## Imports

In [1]:
# SQL Queries
import sqlite3

# Dataframe
import pandas as pd

# Graphing/visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Offline Exercise

This notebook has the questions and instructions for the offline exercise. If you haven't already, please go through the `README.md` file before you begin answering the questions here.

Make sure you do NOT commit your answers back to the repo. Instead, copy-paste this notebook together with the `touches.sqlite3` file to a separate folder in your machine, type your answers in that copied notebook, and when you're done, share the new Jupyter notebook back with the recruiter together with additional modules, graphs etc. that you may have. You would need to use the data in the `touches.sqlite3` file to answer all the questions.

### Question 1. Find the most common industry out of all the companies that were `contacted`.

In [2]:
con = sqlite3.connect("touches.sqlite3")  # Connecting SQL to database
cursor = con.cursor()  # Connecting curor/pointer to database

In [3]:
cursor.execute("""
SELECT MAX(industry) FROM companies c 
INNER JOIN persons p ON c.id = p.company_id
LEFT JOIN touches t ON p.id = t.person_id
WHERE status LIKE "completed"
""")

print(cursor.fetchall())

[('eCommerce',)]


### Code Explanation

Question 1 wants to return the most
common industry that were contacted.

We first use the key word MAX to select
the max value of the industry column from
the table companies.

The two tables, companies and persons connect 
using an id. An inner join is performed to extract
as much information as possible.

The last statement is the WHERE statement.
Showing results where the status column has
a value of completed

### Question 2. What is the most common touch type sellers use when they’re making their first touch with a person? What about first touch with a company?

In [4]:
# Query for common touch type for a person
cursor.execute("""
SELECT MAX(touch_type) FROM touches t 
INNER JOIN persons p ON t.person_id = p.id
""")

print("Common touch type for person:", cursor.fetchall())

# Second Solution
cursor.execute("""
SELECT DISTINCT MAX(touch_type) FROM persons p
INNER JOIN opportunities o ON p.company_id = o.company_id
INNER JOIN companies c ON p.company_id = c.id
INNER JOIN touches t ON p.id = t.person_id
GROUP BY p.id;
""")

print("Common touch type for person Soultion #2:", cursor.fetchall())

Common touch type for person: [('Phone',)]
Common touch type for person Soultion #2: [('Other',), ('Phone',), ('Email',), ('Integration',)]


### Code Explanation

Selecting the max value of the touch_type column from the touches table.

An inner join is then taken place using the persons table; connecting using id.

The end result turns back phone.

**Reasoning:**

I want to take a moment to explain the process behind this code, for this was actually a little confusing to perform:

I combine the two tables, touches and persons because I'm asked to find the most common touch with a person, and the persons table seemed like the correct choice, and I was able to join it with touches.

The main distiction is the Group by

In [5]:
# Query for common touch type with company
cursor.execute("""
SELECT DISTINCT MAX(touch_type) FROM persons p
INNER JOIN opportunities o ON p.company_id = o.company_id
INNER JOIN companies c ON p.company_id = c.id
INNER JOIN touches t ON p.id = t.person_id
GROUP BY o.company_id
""")

print("Commont touch type for company:", cursor.fetchall())

Commont touch type for company: [('Phone',), ('Other',), ('Email',), ('Integration',)]


### Code Explanation

Selecting the max values out of the touch_type column. 

Grabbing the companies table and performing an inner join with persons on company id and id.

A second inner join is performed with opportunites on the company table id and the opportunities company_id.

The last join is a left join with touches, where we get the touch_type column, and this is on persons id.

**Reasoning:**

I want to take a moment to show my thought process:

I chose to start with the companies table and combine all of the tables with it to achieve my result. 

I chose the companies table because of the question that is asked. 

### Question 3. Describe the distribution of the job seniorities of people that a seller will first try to contact within a company.

To answer this question, you may use visuals, graphs, bunch of scores, tables, writeups - whatever you want. We literally want you to "describe" the distribution to us in the best way you can!

(Note: this question doesn't really have one right answer. It's more about your style of communicating the results.)

In [6]:
cursor.execute("""
SELECT * FROM persons p
INNER JOIN opportunities o ON p.company_id = o.company_id
INNER JOIN companies c ON p.company_id = c.id
INNER JOIN touches t ON p.id = t.person_id
GROUP BY p.company_id
""")

# Converting SQL Query to a dataframe
touch_df = pd.DataFrame(cursor.fetchall())
touch_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,82,1,Director,3601,1,2018-03-12 14:08:25.107017,Closed Won,1,Diversified Telecommunication Services,51-200 employees,US/Eastern,166380,82,2017-06-01,completed,Phone
1,126,2,Executive,3971,2,2018-03-28 18:52:27.102803,Closed Lost,2,Internet Software & Services,51-200 employees,US/Mountain,335104,126,2018-01-24,completed,Other
2,511,4,Manager,5655,4,2018-06-12 16:54:57.192067,Closed Won,4,Internet Software & Services,,US/Pacific,813354,511,2019-02-22,completed,Other
3,548,6,Manager,1775,6,2018-02-12 13:29:30.725185,Closed Lost,6,Diversified Financial Services,11-50 employees,US/Pacific,217807,548,2017-08-27,completed,Other
4,578,8,Individual Contributor,3339,8,2018-02-23 20:51:26.130273,Closed Lost,8,Telecommunications,,US/Eastern,542182,578,2018-07-24,completed,Other


In [7]:
# Adding column names to dataframe
touch_df.rename(columns={0: "person_id", 1: "company_id", 2: "job_seniority", 3:"opportunity_table_id", 
                         4: "company_id_duplicate", 5: "created_at", 6: "latest_stage", 7: "companies_table_id_duplicate", 8: "industry", 
                         9: "size", 10: "location", 11: "touches_table_id", 12: "person_id_duplicate", 13: "touch_scheduled_on", 
                         14: "status", 15: "touch_type"}, inplace=True)

In [8]:
touch_df.head()

,person_id,company_id,job_seniority,opportunity_table_id,company_id_duplicate,created_at,latest_stage,companies_table_id_duplicate,industry,size,location,touches_table_id,person_id_duplicate,touch_scheduled_on,status,touch_type
0,82,1,Director,3601,1,2018-03-12 14:08:25.107017,Closed Won,1,Diversified Telecommunication Services,51-200 employees,US/Eastern,166380,82,2017-06-01,completed,Phone
1,126,2,Executive,3971,2,2018-03-28 18:52:27.102803,Closed Lost,2,Internet Software & Services,51-200 employees,US/Mountain,335104,126,2018-01-24,completed,Other
2,511,4,Manager,5655,4,2018-06-12 16:54:57.192067,Closed Won,4,Internet Software & Services,,US/Pacific,813354,511,2019-02-22,completed,Other
3,548,6,Manager,1775,6,2018-02-12 13:29:30.725185,Closed Lost,6,Diversified Financial Services,11-50 employees,US/Pacific,217807,548,2017-08-27,completed,Other
4,578,8,Individual Contributor,3339,8,2018-02-23 20:51:26.130273,Closed Lost,8,Telecommunications,,US/Eastern,542182,578,2018-07-24,completed,Other


In [9]:
touch_df.shape

(4971, 16)

In [10]:
# Dropping duplicated columns on axis=1, columns.
touch_df.drop(touch_df.columns[[4, 7, 12]], axis=1, inplace=True)

In [11]:
touch_df.head()

,person_id,company_id,job_seniority,opportunity_table_id,created_at,latest_stage,industry,size,location,touches_table_id,touch_scheduled_on,status,touch_type
0,82,1,Director,3601,2018-03-12 14:08:25.107017,Closed Won,Diversified Telecommunication Services,51-200 employees,US/Eastern,166380,2017-06-01,completed,Phone
1,126,2,Executive,3971,2018-03-28 18:52:27.102803,Closed Lost,Internet Software & Services,51-200 employees,US/Mountain,335104,2018-01-24,completed,Other
2,511,4,Manager,5655,2018-06-12 16:54:57.192067,Closed Won,Internet Software & Services,,US/Pacific,813354,2019-02-22,completed,Other
3,548,6,Manager,1775,2018-02-12 13:29:30.725185,Closed Lost,Diversified Financial Services,11-50 employees,US/Pacific,217807,2017-08-27,completed,Other
4,578,8,Individual Contributor,3339,2018-02-23 20:51:26.130273,Closed Lost,Telecommunications,,US/Eastern,542182,2018-07-24,completed,Other


In [12]:
touch_df.shape

(4971, 13)

### Question 4. Describe the distribution of the mixture of job seniorities of people that a seller will touch during the entire engagement with a company.
Keep in mind that you get to decide what “mixture” means, so do begin your answer by defining it - and explaining why you think this definition makes sense. Again, feel free to use whatever visuals, graphs, bunch of scores, tables, writeups etc. that you think is appropriate for this question.

(Note: this question doesn't really have one right answer. It's more about your style of communicating the results.)

### Question 5. Build an algorithm that, when a seller is about to contact a company it never reached out to before, recommends the best/optimal mixture of job seniorities (with “mixture” as defined by you in Question 4.) they should be targeting within that company during the entire engagement.

This is the hardest question in this exercise. If you feel you have a good idea what the algorithm should look like but either feel uncomfortable coding-wise or feel pressed for time, it is perfectly OK to describe your idea in words. Be clear, explain both what it does and why you think this is the right approach.

Whether or not you go the code-route or writeup-route, be sure to explain the assumptions and choices you made, why you made them and how you would test them.

(Note: like with the previous two questions, this question doesn't really have one right answer either!)